In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
data = []
labels = []
classes = 43
cur_path = os.getcwd()

Using TensorFlow backend.


In [2]:
cur_path

'D:\\Siddhi\\Projects\\sign-detection'

# Loading the images and creating the target variable 

In [3]:
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape)

(39209, 30, 30, 3) (39209,)


# Splitting into train and test set

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

# Model training with 2 cnvolutional layers

In [5]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))
model.save("my_model.h5")

Train on 31367 samples, validate on 7842 samples
Epoch 1/15
31367/31367 [==============================] - 101s 3ms/step - loss: 2.1194 - accuracy: 0.4487 - val_loss: 0.6621 - val_accuracy: 0.8379
Epoch 2/15
31367/31367 [==============================] - 101s 3ms/step - loss: 0.8922 - accuracy: 0.7363 - val_loss: 0.3230 - val_accuracy: 0.9086
Epoch 3/15
31367/31367 [==============================] - 100s 3ms/step - loss: 0.5685 - accuracy: 0.8292 - val_loss: 0.2371 - val_accuracy: 0.9283
Epoch 4/15
31367/31367 [==============================] - 101s 3ms/step - loss: 0.4407 - accuracy: 0.8653 - val_loss: 0.1524 - val_accuracy: 0.9552
Epoch 5/15
31367/31367 [==============================] - 100s 3ms/step - loss: 0.3538 - accuracy: 0.8938 - val_loss: 0.1271 - val_accuracy: 0.9637
Epoch 6/15
31367/31367 [==============================] - 101s 3ms/step - loss: 0.3347 - accuracy: 0.9006 - val_loss: 0.1221 - val_accuracy: 0.9629
Epoch 7/15
31367/31367 [==============================] - 101s 

# Loading test set and reshaping

In [6]:
from sklearn.metrics import accuracy_score
y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)



# Predict the class of the test set images

In [7]:
pred = model.predict_classes(X_test)
pred[:1]

array([16], dtype=int64)

# Calculating accuracy of trained model

In [8]:
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))
model.save("traffic_classifier.h5")

0.9436262866191607


# Load the trained model

In [9]:
import numpy
from keras.models import load_model
model = load_model('traffic_classifier.h5')

In [10]:
#Defined classes
classes = { 1:'Speed limit (20km/h)',
            2:'Speed limit (30km/h)', 
            3:'Speed limit (50km/h)', 
            4:'Speed limit (60km/h)', 
            5:'Speed limit (70km/h)', 
            6:'Speed limit (80km/h)', 
            7:'End of speed limit (80km/h)', 
            8:'Speed limit (100km/h)', 
            9:'Speed limit (120km/h)', 
            10:'No passing', 
            11:'No passing veh over 3.5 tons', 
            12:'Right-of-way at intersection', 
            13:'Priority road', 
            14:'Yield', 
            15:'Stop', 
            16:'No vehicles', 
            17:'Veh > 3.5 tons prohibited', 
            18:'No entry', 
            19:'General caution', 
            20:'Dangerous curve left', 
            21:'Dangerous curve right', 
            22:'Double curve', 
            23:'Bumpy road', 
            24:'Slippery road', 
            25:'Road narrows on the right', 
            26:'Road work', 
            27:'Traffic signals', 
            28:'Pedestrians', 
            29:'Children crossing', 
            30:'Bicycles crossing', 
            31:'Beware of ice/snow',
            32:'Wild animals crossing', 
            33:'End speed + passing limits', 
            34:'Turn right ahead', 
            35:'Turn left ahead', 
            36:'Ahead only', 
            37:'Go straight or right', 
            38:'Go straight or left', 
            39:'Keep right', 
            40:'Keep left', 
            41:'Roundabout mandatory', 
            42:'End of no passing', 
            43:'End no passing veh > 3.5 tons' }

In [11]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from PIL import ImageTk, Image
 
# load an image and predict the class
def predict(file):
    #img = load_image('sample-img.png')
    image = Image.open(file)
    image = image.resize((30,30))
    image = numpy.expand_dims(image, axis=0)
    image = numpy.array(image)
    pred = model.predict_classes([image])[0]
    sign = classes[pred+1]
    print(sign)
    #digit = model.predict_classes(img)
    #print(digit[0])
 

predict('sample-img.png')

Turn right ahead
